<div style='background-color: #87ceeb;
    border: 0.5em solid black;
    border-radius: 0.5em;
    padding: 1em;'>
    <h2>Activité</h2>
    <h1>Sortie du labyrinthe</h1>
</div>

On donne la définition d'une classe `Pile`.

In [ ]:
class Pile:
    def __init__(self):
        self.liste = []
    
    def est_vide(self):
        return self.liste == []

    def empiler(self, elem):
        self.liste.append(elem)
    
    def depiler(self):
        if self.est_vide():
            raise ValueError("la pile est vide")
        return self.liste.pop()
    
    def __str__(self):
        chaine = '|'
        for k in range(len(self.liste)):
            chaine += f" {self.liste[k]} |"
        return chaine

### Représentation d'un labyrinthe en mémoire et à l'écran

On représente un labyrinthe par une grille rectangulaire dans laquelle les murs sont des cases noires et les passages des cases blanches. La grille est représentée en mémoire par un tableau `grille` à deux dimensions dont les éléments sont `O` pour un passage et `1` pour un mur.

<img src='https://ntoulzac.github.io/Cours-NSI-Terminale/piles_files/images/laby1.png' width='450px'>

**(1)** Définir `grille` pour le labyrinthe dessiné ci-dessus.

In [ ]:
grille = [[1, 1, 1, 1, 1, 0, 1, 1, 1],
          [0, 0, 0, 0, 1, 0, 1, 0, 1],
          [1, 0, 1, 0, 1, 0, 1, 0, 1],
          [1, 0, 1, 0, 1, 0, 0, 0, 1],
          [1, 0, 1, 1, 1, 1, 1, 0, 1],
          [1, 0, 0, 0, 0, 0, 0, 0, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1]]

On repère la position de chaque case par un p-uplet de la forme `(L, C)`, où `L` et `C` sont respectivement un numéro de ligne et de colonne.

**(2)** Définir deux p-uplets `entree` et `sortie` correspondant aux positions de l'entrée et de la sortie du labyrinthe dessiné ci-dessus.

In [ ]:
entree = (1, 0)
sortie = (0, 5)

**(3)** Définir l'ébauche d'une classe `Labyrinthe` dont la méthode constructeur prend trois paramètres d'entrée : un tableau à deux dimensions (contenant le plan des passages et des murs) et deux p-uplets (position de l'entrée et de la sortie). Les attributs des instances de la classe `Labyrinthe` sont :
- `grille` : un tableau à deux dimensions contenant le plan des passages et des murs,
- `nb_l` : un entier correspondant au nombre de lignes de la grille,
- `nb_c` : un entier correspondant au nombre de colonnes de la grille,
- `entree` : un p-uplet correspondant à la position de l'entrée du labyrinthe,
- `sortie` : un p-uplet correspondant à la position de la sortie du labyrinthe.

In [ ]:
class Labyrinthe:
    def __init__(self, tab, posE, posS):
        self.grille = tab
        self.nb_l = len(self.grille)
        self.nb_c = len(self.grille[0])
        self.entree = posE
        self.sortie = posS

**(4)** Après l'avoir téléchargé [ici](https://ntoulzac.github.io/Cours-NSI-Terminale/piles_files/fichiers/fenetre_laby.py) et placé dans le même répertoire que ce carnet _Jupyter_, importer le module `fenetre_laby` sous le nom `fl`.

In [ ]:
import fenetre_laby as fl

**(5)** Afficher l'aide de la classe `FenetreLaby` définie dans ce même module.

In [ ]:
help(fl.FenetreLaby)

**(6)** Créer une instance de la classe `Labyrinthe` correspondant au labyrinthe dessiné plus haut, puis faire afficher ce labyrinthe dans une fenêtre `pygame`.

In [ ]:
nouvelle_grille = grille[:][:]
mon_laby = Labyrinthe(nouvelle_grille, entree, sortie)
ma_fenetre = fl.FenetreLaby(mon_laby)
ma_fenetre.actualiser(mon_laby)

**(7)** Fermer la fenêtre `pygame`.

In [ ]:
ma_fenetre.fermer()

### Utilisation d'une pile pour rechercher la sortie du labyrinthe

On souhaite maintenant pouvoir trouver comment atteindre la sortie du labyrinthe à partir de l'entrée.

**(8)** Recopier ci-dessous la définition de la classe `Labyrinthe` en y ajoutant :
- une méthode `visiter_case`. Cette méthode prend en paramètre d'entrée un p-uplet `pos` de la forme `(L, C)` et une instance de la classe `FenetreLaby`. Elle remplace sur la ligne `L` et dans la colonne `C` de la `grille` le nombre `0` (case non encore visitée) par le nombre `2` (case déjà visitée). De plus, elle colorie en rouge la case en question, attend un quart de seconde puis la colorie en bleu.
- une méthode `voisines_non_visitees`. Cette méthode prend en paramètre d'entrée un p-uplet `pos` de la forme `(L, C)` et elle revoie le tableau composé des positions des cases qui sont voisines de `pos` (à droite, au dessus, au dessous, à gauche) et qui n'ont pas encore été visitées.

**(9)** Compléter le code de la méthode `parcourir`.

In [ ]:
from time import sleep

class Labyrinthe:
    def __init__(self, tab, posE, posS):
        self.grille = tab
        self.nb_l = len(self.grille)
        self.nb_c = len(self.grille[0])
        self.entree = posE
        self.sortie = posS
        
    def visiter_case(self, pos, fen):
        (L, C) = pos
        self.grille[L][C] = 2
        fen.afficher_case(pos, fl.ROUGE)
        sleep(0.25)
        fen.afficher_case(pos, fl.BLEU)
    
    def voisines_non_visitees(self, pos):
        (L, C) = pos
        voisines = []
        for (a, b) in [(0, 1), (-1, 0), (1, 0), (0, -1)]:
            if 0 <= L+a < self.nb_l and 0 <= C+b < self.nb_c:
                if self.grille[L+a][C+b] == 0:
                    voisines.append((L+a, C+b))
        return voisines
    
    def parcourir(self, fenetre):
        (L, C) = self.entree  # On se place à l'entrée du labyrinthe.
        pile = Pile()  # On crée une pile vide.
        continuer_parcours = True
        while continuer_parcours:
            self.visiter_case((L, C), fenetre)  # On visite la case (L, C).
            if (L, C) == self.sortie:  # Si c'est la sortie alors...
                pile.empiler((L, C))  # ... on empile la position (L, C)...
                continuer_parcours = False  # ... et le parcours se termine sur un succès.
            else:  # Sinon...
                voisines = self.voisines_non_visitees((L, C))  # ... on liste les voisines de (L, C) non visitées.
                if voisines == []:  # S'il n'y en a aucune...
                    if pile.est_vide():  # ... et que la pile est vide alors...
                        continuer_parcours = False  # ... le parcours se termine sur un échec...
                    else:
                        (L, C) = pile.depiler()  # ... sinon on dépile un élément pour revenir en arrière.
                else:  # Si, par contre, il y a des voisines non visitées alors...
                    pile.empiler((L, C))  # on empile la position (L, C)...
                    (L, C) = voisines[0]  # et on remplace (L, C) par une de ses voisines non visitées.
        return pile

In [ ]:
from copy import deepcopy
nouvelle_grille = deepcopy(grille)

mon_laby = Labyrinthe(nouvelle_grille, entree, sortie)
ma_fenetre = fl.FenetreLaby(mon_laby)
ma_fenetre.actualiser(mon_laby)
mon_laby.parcourir(ma_fenetre)
ma_fenetre.fermer()

<div class='rq'>La fonction <code>deepcopy</code> du module <code>copy</code> permet de créer une copie du tableau à deux dimensions <code>grille</code>, et donc de ne pas modifier <code>grille</code> au cours du parcours du labyrinthe.</div>

### Approfondissement

On suppose maintenant que le plan du labyrinthe est stocké dans un fichier texte, comme dans les fichiers du répertoire `Grilles`. L'entrée du labyrinthe y est représentée par un caractère `E` et la sortie par un `S`.

**(10)** Définir une méthode `lire_fichier` prenant en paramètre d'entrée un `nom_de_fichier` sous forme de chaine de caractères et initialisant la `grille` et les p-uplets `entree` et `sortie` à partir du fichier.

In [ ]:
class Labyrinthe:
    def __init__(self, nom_de_fichier):
        self.lire_fichier(nom_de_fichier)
        self.nb_l = len(self.grille)
        self.nb_c = len(self.grille[0])

    def lire_fichier(self, nom_de_fichier):
        with open(nom_de_fichier, 'r') as fichier:
            self.grille = []
            L = 0
            for chaine in fichier:
                ligne = []
                C = 0
                for car in chaine:
                    if car == 'E':
                        ligne.append(0)
                        self.entree = (L, C)
                    elif car == 'S':
                        ligne.append(0)
                        self.sortie = (L, C)
                    elif car == '0' or car == '1':
                        ligne.append(int(car))
                    C += 1
                self.grille.append(ligne)
                L += 1
    
    def visiter_case(self, pos, fen):
        (L, C) = pos
        self.grille[L][C] = 2
        fen.afficher_case(pos, fl.ROUGE)
        fen.afficher_case(pos, fl.BLEU)
    
    def voisines_non_visitees(self, pos):
        (L, C) = pos
        voisines = []
        for (a, b) in [(0, 1), (-1, 0), (1, 0), (0, -1)]:
            if 0 <= L+a < self.nb_l and 0 <= C+b < self.nb_c:
                if self.grille[L+a][C+b] == 0:
                    voisines.append((L+a, C+b))
        return voisines
    
    def parcourir(self, fenetre):
        (L, C) = self.entree  # On se place à l'entrée du labyrinthe.
        pile = Pile()  # On crée une pile vide.
        continuer_parcours = True
        while continuer_parcours:
            self.visiter_case((L, C), fenetre)  # On visite la case (L, C).
            if (L, C) == self.sortie:  # Si c'est la sortie alors...
                pile.empiler((L, C))  # ... on empile la position (L, C)...
                continuer_parcours = False  # ... et le parcours se termine sur un succès.
            else:  # Sinon...
                voisines = self.voisines_non_visitees((L, C))  # ... on liste les voisines de (L, C) non visitées.
                if voisines == []:  # S'il n'y en a aucune...
                    if pile.est_vide():  # ... et que la pile est vide alors...
                        continuer_parcours = False  # ... le parcours se termine sur un échec...
                    else:
                        (L, C) = pile.depiler()  # ... sinon on dépile un élément pour revenir en arrière.
                else:  # Si, par contre, il y a des voisines non visitées alors...
                    pile.empiler((L, C))  # on empile la position (L, C)...
                    (L, C) = voisines[0]  # et on remplace (L, C) par une de ses voisines non visitées.
        return pile                

In [ ]:
import fenetre_laby as fl

In [ ]:
def resoudre_laby(nom_de_fichier):
    laby = Labyrinthe(nom_de_fichier)
    fenetre = fl.FenetreLaby(laby)
    fenetre.actualiser(laby)
    pile = laby.parcourir(fenetre)
    while not pile.est_vide():
        pos = pile.depiler()
        fenetre.afficher_case(pos, fl.ROUGE)
    fenetre.fermer()

Lorsque le parcours du labyrinthe est terminé, on dépile toutes les positions stockées dans la `pile` et on affiche en rouge les cases correspondantes, faisant ainsi apparaître à l'écran le chemin qui mène de l'entrée à la sortie.

Vous pouvez tester votre code avec les grilles téléchargeables [ici](https://ntoulzac.github.io/Cours-NSI-Terminale/piles_files/fichiers/grilles.zip).